In [44]:
import os
import sys
import json
import time
import numpy as np
from sklearn.cluster import KMeans
from gensim.models import Word2Vec


In [64]:
import copy

In [127]:
index1 = json.load(open('../data/sen_index_1.json'))
index2 = json.load(open('../data/sen_index_2.json'))
index3 = json.load(open('../data/sen_index_3.json'))
index4 = json.load(open('../data/sen_index_4.json'))

In [128]:
index1.update(index2)
index1.update(index3)
index1.update(index4)
index = copy.copy(index1)

In [132]:
n_cluster = 5
X = np.load('../data/PCA/sen_vector_PCA.npy')
kmeans = KMeans(n_clusters=n_cluster,max_iter=10000).fit(X)
pred_label = kmeans.predict(X)


In [46]:
#####
ref = json.load(open('../data/preprocessing/standard_ref/standard_ref_without_effect.json'))

sentences = json.load(open('../data/feature_extraction/entity_pair_sentences.json'))

entity_names = open('../data/clean_data/entity_total.csv',encoding='utf8').read().split('\n')[:-1]

#####
path = '../data/model/noun_verb/300/model'
model = Word2Vec.load(path)
voca = list(model.wv.vocab)
model = ''


In [133]:
entity_dict = {}
for row in entity_names :
    entity_dict[row.split('\t')[1]] = row.split('\t')[0]

In [134]:
ok = 0
total = 0
ref_pair = []
for herb in ref :
    for drug in ref[herb] :
        if entity_dict[herb] in voca and entity_dict[drug] in voca :
            ref_pair.append([entity_dict[drug],entity_dict[herb]])


In [135]:
entity_pair = []
for each in sentences :
    entity_pair.append(sorted([each[1][each[0][0]],each[1][each[0][1]]]))

In [136]:
len(ref_pair)

143

In [149]:
count = 0
result = {}
top = []
for each in ref_pair :
    if each in entity_pair :
        top.append(entity_pair.index(each))
        group = pred_label[entity_pair.index(each)]
        #if group == 9 :
            #print(each)
        try :
            result[group]+=1
        except :
            result[group] = 1
print(result)

{2: 19, 1: 2, 3: 1, 0: 1}


In [138]:
from scipy import spatial

##### cluster labeling - cosine similarity
centroids = kmeans.cluster_centers_
res = {}

for i in range(len(pred_label)) :
    cosine = 1 - spatial.distance.cosine(X[i],centroids[pred_label[i]] )
    if pred_label[i] not in res :
        res[pred_label[i]] = {cosine:i}
    else :
        res[pred_label[i]][cosine] = i


In [139]:
count = {}
for each in pred_label :
    if each in count :
        count[each]+=1
    else:
        count[each] = 1
print([count[i] for i in list(sorted(count)) ])

[8809, 2600, 15402, 1710, 3396]


In [250]:
top =[ res[2][i] for i in sorted(res[2])[::-1][0:50] ]
cheating = {}
for i in top :
    if str(i) in index  :
        ab_dict = json.load(open('../data/preprocessing/abstract_extract_filtered_tracked/%s'%index[str(i)][0])) 
        print(top.index(i))
        #print(pred_label[i])
        #print(index[str(i)][1])
        print(ab_dict[index[str(i)][1]])
        print(sentences[i][1])
        ''''''
        for word in sentences[i][1][sentences[i][0][0]:sentences[i][0][1]] :
            try :
                cheating[word]+=1
            except :
                cheating[word] = 1

for each in list(cheating):
    if cheating[each] < 10 :
        cheating.pop(each)
        ''''''

0
in order to find new tyrosinase inhibitors and antioxidant materials, we investigated 44 plants, which were evaluated for the anti-tyrosinase and antioxidant activities. the mushroom tyrosinase inhibition assay and 2, 2-diphenyl-1- picrylhydrazyl (dpph) radical scavenging assay were conducted to evaluate these activities. among all tested plant extracts, morus alba l. (positive control), rhodiola crenulata (hook. f. et thoms.) h. ohba, momordica charantia l., cuminum cyminum l. et al. exhibit higher tyrosinase inhibition. rhodiola crenulata (hook. f. et thoms.) h. ohba, rosa rugosa thunb. and eugenia caryophyllata thunb. perform the highest antioxidant activity, similar to vitamin c (the positive control). a low positive correlation is found in the dpph radical scavenging and tyrosinase inhibition assay. considering these factors, the extracts of rhodiola crenulata (hook. f. et thoms.) h. ohba, alpinia officinarum hance and zanthoxylum bungeanum maxim. exhibit high anti-tyrosinase an

In [248]:
cheating

{'activ': 12,
 'cell': 36,
 'combin': 10,
 'drug_entity_3194': 14,
 'express': 16,
 'herb_entity_4798': 11,
 'protein': 21,
 'studi': 15,
 'use': 20}

In [212]:
clusters = {}
for i in range(len(pred_label)) :
    try :
        clusters[pred_label[i]].append(sentences[i])
    except :
        clusters[pred_label[i]] = [sentences[i]]


In [ ]:
[8809, 2600, 15402, 1710, 3396]

In [214]:
clusters_dict = {}
clusters_dict[1] = clusters[0]
clusters_dict[3] = clusters[1]
clusters_dict[4] = clusters[2]
clusters_dict[2] = clusters[3]
clusters_dict[0] = clusters[4]


In [233]:
keydict ={ 4 : ['use','inhibi','increas','show'], 0:['grown','root'],1:['express','combin','protein','cell'],2:['',''],3:['','']}
check_cluster(clusters_dict[3],keys)

9.0


In [222]:
def check_cluster(sentences,keys):
    total = len(sentences)
    count = 0
    for each in sentences :
        sentence = each[1]
        if len(set(sentence[each[0][0]:each[0][1]]).intersection(keys)) != 0 :
            count +=1
    print((float(count)/total)*100)

In [77]:
def daviesbouldin(X, labels, centroids):

    import numpy as np
    from scipy.spatial.distance import pdist, euclidean

    nbre_of_clusters = len(centroids) #Get the number of clusters
    distances = [[] for e in range(nbre_of_clusters)] #Store intra-cluster distances by cluster
    distances_means = [] #Store the mean of these distances
    DB_indexes = [] #Store Davies_Boulin index of each pair of cluster
    second_cluster_idx = [] #Store index of the second cluster of each pair
    first_cluster_idx = 0 #Set index of first cluster of each pair to 0

    # Step 1: Compute euclidean distances between each point of a cluster to their centroid
    for cluster in range(nbre_of_clusters):
        for point in range(X[labels == cluster].shape[0]):
            distances[cluster].append(euclidean(X[labels == cluster][point], centroids[cluster]))

    # Step 2: Compute the mean of these distances
    for e in distances:
        distances_means.append(np.mean(e))

    # Step 3: Compute euclidean distances between each pair of centroid
    ctrds_distance = pdist(centroids) 

    # Tricky step 4: Compute Davies-Bouldin index of each pair of cluster   
    for i, e in enumerate(e for start in range(1, nbre_of_clusters) for e in range(start, nbre_of_clusters)):
        second_cluster_idx.append(e)
        if second_cluster_idx[i-1] == nbre_of_clusters - 1:
            first_cluster_idx += 1
        DB_indexes.append((distances_means[first_cluster_idx] + distances_means[e]) / ctrds_distance[i])

    # Step 5: Compute the mean of all DB_indexes   
    print("DAVIES-BOULDIN Index: %.5f" % np.mean(DB_indexes)) 
    return np.mean(DB_indexes)

In [78]:
davie = {}
calinski = {}
X = np.load('../data/PCA/sen_vector_PCA.npy')
for n_cluster in range(2,50) :
    kmeans = KMeans(n_clusters=n_cluster, max_iter=1000).fit(X)
    labels = kmeans.predict(X)
    centroids = kmeans.cluster_centers_
    davie[n_cluster] = daviesbouldin(X, labels, centroids)
    calinski[n_cluster] =  sklearn.metrics.calinski_harabaz_score(X,labels)
    

DAVIES-BOULDIN Index: 2.15218
DAVIES-BOULDIN Index: 1.57891
DAVIES-BOULDIN Index: 1.64996
DAVIES-BOULDIN Index: 1.67187
DAVIES-BOULDIN Index: 1.68358
DAVIES-BOULDIN Index: 1.23870
DAVIES-BOULDIN Index: 1.25308
DAVIES-BOULDIN Index: 1.28893
DAVIES-BOULDIN Index: 1.28271
DAVIES-BOULDIN Index: 1.30754
DAVIES-BOULDIN Index: 1.33104
DAVIES-BOULDIN Index: 1.32987
DAVIES-BOULDIN Index: 1.34020
DAVIES-BOULDIN Index: 1.32118
DAVIES-BOULDIN Index: 1.32412
DAVIES-BOULDIN Index: 1.33698
DAVIES-BOULDIN Index: 1.33346
DAVIES-BOULDIN Index: 1.34397
DAVIES-BOULDIN Index: 1.34838
DAVIES-BOULDIN Index: 1.34047
DAVIES-BOULDIN Index: 1.32625
DAVIES-BOULDIN Index: 1.32545
DAVIES-BOULDIN Index: 1.32743
DAVIES-BOULDIN Index: 1.31306
DAVIES-BOULDIN Index: 1.32498
DAVIES-BOULDIN Index: 1.29528
DAVIES-BOULDIN Index: 1.32168
DAVIES-BOULDIN Index: 1.29037
DAVIES-BOULDIN Index: 1.31982
DAVIES-BOULDIN Index: 1.28982
DAVIES-BOULDIN Index: 1.29385
DAVIES-BOULDIN Index: 1.27049
DAVIES-BOULDIN Index: 1.28276
DAVIES-BOU